<a href="https://colab.research.google.com/github/annikarichardson2024/Project1/blob/main/SQL_Connect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyMySQL
!pip install mysql-connector-python
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 24.4 MB/s eta 0:00:00


In [2]:
host_name = "datatbase.ds2002.org"
#host_ip = "127.0.0.1"
port = "3306"

user_id = "aer6bk"
pwd = "aer6bk!"
db_name = "sakila"

In [8]:
import pandas as pd
import os
import pymysql
import warnings

conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM actor;')

    for row in cursor.fetchmany(size=10):
        print(row)

    cursor.close()

except:
    print ("Error: unable to fetch data")

conn.close()

(1, 'PENELOPE', 'GUINESS', datetime.datetime(2006, 2, 15, 4, 34, 33))
(2, 'NICK', 'WAHLBERG', datetime.datetime(2006, 2, 15, 4, 34, 33))
(3, 'ED', 'CHASE', datetime.datetime(2006, 2, 15, 4, 34, 33))
(4, 'JENNIFER', 'DAVIS', datetime.datetime(2006, 2, 15, 4, 34, 33))
(5, 'JOHNNY', 'LOLLOBRIGIDA', datetime.datetime(2006, 2, 15, 4, 34, 33))
(6, 'BETTE', 'NICHOLSON', datetime.datetime(2006, 2, 15, 4, 34, 33))
(7, 'GRACE', 'MOSTEL', datetime.datetime(2006, 2, 15, 4, 34, 33))
(8, 'MATTHEW', 'JOHANSSON', datetime.datetime(2006, 2, 15, 4, 34, 33))
(9, 'JOE', 'SWANK', datetime.datetime(2006, 2, 15, 4, 34, 33))
(10, 'CHRISTIAN', 'GABLE', datetime.datetime(2006, 2, 15, 4, 34, 33))


In [7]:
# API CALL
import os
import json
import pprint
import requests
import requests.exceptions
import pandas as pd


def get_api_response(url, response_type):
    try:
        response = requests.get(url)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    if response_type == 'json':
        result = json.dumps(response.json(), sort_keys=True, indent=4)
    elif response_type == 'dataframe':
        result = pd.json_normalize(response.json())
    else:
        result = "An unhandled error has occurred!"

    return result

url = " insert new data set or data frame?"

response_type = ['json', 'dataframe']

json_string = get_api_response(url, response_type[0])
print(json_string)

An Unknown Error occurred: MissingSchema("Invalid URL 'insert new data set or data frame?': No scheme supplied. Perhaps you meant https://insert new data set or data frame??")
